In [69]:
'''Trains a simple convnet on the face landmark dataset.
Adapted from Keras MNIST CNN example code.
'''

import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils


batch_size = 10
nb_landmarks = 5
nb_epoch = 500
nb_verbose = 0
#nb_validation_data = (X_test, _Y_test)
nb_validation_data = None

# input image dimensions
img_chns, img_rows, img_cols = 3, 20, 20
# number of convolutional filters to use
nb_filters = [32, 64]
# size of pooling area for max pooling
nb_pool_sizes = [(2, 2), (2, 2)]
# convolution kernel size
nb_conv = 3
# activator
nb_activator = 'tanh'
# number of fully connected neurons in the penultimate layer
nb_penu_neurons = 256
# size of output vector, four "corner values" for each landmark
nb_output_size = nb_landmarks * 4

input_img = Input(shape=(img_chns, img_rows, img_cols), dtype='float32', name='input_img')

x = input_img
x = (Convolution2D(nb_filters[0], nb_conv, nb_conv))(x)
x = (Activation(nb_activator))(x)
x = (MaxPooling2D(pool_size=nb_pool_sizes[0]))(x)
x = (Convolution2D(nb_filters[1], nb_conv, nb_conv))(x)
x = (Activation(nb_activator))(x)
x = (MaxPooling2D(pool_size=nb_pool_sizes[1]))(x)
x = (Dropout(0.25))(x)

x = (Flatten())(x)
x = (Dense(nb_penu_neurons))(x)
x = (Activation(nb_activator))(x)
x = (Dropout(0.5))(x)
output_landmarks = [Activation('sigmoid')((Dense(4, name='landmark-%d'%(_)))(x)) for _ in range(nb_landmarks)]

model = Model(input=input_img, output=output_landmarks)

model.compile(loss='categorical_crossentropy',
              loss_weights=[1, 1, 5, 1, 1],
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, _Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=nb_verbose, validation_data=nb_validation_data)

score = model.evaluate(X_test, _Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 11.5112075555
Test accuracy: 0.98126171143


In [70]:
# save model
model_filename = '../model/corners-all(ratio-0.8,rand-1337),20px,500ep'
print(model.to_json(), file=open(model_filename+'.json', 'w'))
model.save_weights(model_filename+'.weight')

In [71]:
_Y_pred = model.predict(X_test)
Y_pred = Y_model_to_data(_Y_pred)

In [73]:
# observe learning results
import random
y_converter = lambda y : np.array(list(map(corners_to_coord, y)))

plt.figure()
for iplt, i in enumerate(random.sample(range(len(Y_test)), 30)) :
    plt.subplot(5, 6, iplt+1)
    img = X_test[i].transpose((1,2,0))
    io.imshow(img)
    pts2 = y_converter(Y_test[i])
    plt.plot(pts2[:,0]*img_rows, pts2[:,1]*img_cols, 'o')
    pts = y_converter(Y_pred[i])
    plt.plot(pts[:,0]*img_rows, pts[:,1]*img_cols, 'ro')
    plt.axis('off')
plt.tight_layout()
io.show()

In [41]:
def Y_data_to_model(Y) :
    return list(np.transpose(Y, (1,0,2)))
def Y_model_to_data(Y) :
    return np.transpose(np.array(Y), (1,0,2))

In [1]:
from face_data import *

X shape: (8003, 3, 20, 20)
Y shape: (8003, 5, 2)


In [17]:
def coord_to_corners(coord) :
    x, y = coord
    return np.array([(1-x)*(1-y), (1-x)*y, x*y, x*(1-y)])
def corners_to_coord(corners) :
    x0y0, x0y1, x1y1, x1y0 = corners
    return np.array([x1y1+x1y0, x1y1+x0y1]) / sum(corners)

In [3]:
Y = np.array([[coord_to_corners(p) for p in pts] for pts in Y])

In [4]:
(X_train, Y_train), (X_test, Y_test) = split_data(X, Y, ratio_train=0.8, rand_seed=1337)

X_train shape: (6402, 3, 20, 20)
Y_train shape: (6402, 5, 4)
6402 train samples
1601 test samples


In [42]:
_Y_train = Y_data_to_model(Y_train)
_Y_test = Y_data_to_model(Y_test)

In [60]:
Y_pred[0]

array([[ -8.30362225,   8.62354279,   7.49342823,  -7.54829741],
       [ 15.6356926 ,   4.24076223,  10.00317478,  34.21366882],
       [ 23.66015053,  18.45369911,  18.67684364,  23.69833374],
       [-15.26077366, -26.44235802, -13.85803699,  -7.93753767],
       [ -8.21008778, -13.83751297, -27.69595337, -16.66737747]])